In [1]:
import json
from tqdm import tqdm

In [2]:
with open('data_processed.json') as f:
    data = json.loads(f.read())
with open('data_processed_full-list.json') as f:
    data_list = json.loads(f.read())
with open('data_processed_full-list-ids.json') as f:
    data_list_id = json.loads(f.read())

In [3]:
with open('category_dependency.jsonl') as f:
    categories = [ json.loads(line) for line in f.readlines() ] 

In [4]:
with open('child_to_index.json') as f:
    child_to_index = json.loads(f.read())

In [5]:
parents = {}
def recurse(node , vis):
    queue = [(node,0)]
    ret = []
    vis = set()
    while len(queue):
        node = queue.pop(0)
        node , dep = node[0] , node[1] + 1
        if node in vis : continue
        vis |= set([node])
        if node in child_to_index:
            _pars = categories[child_to_index[node]]['parents']
        else : continue
        ret.append((node,dep))
        if dep > 7 : break
        for par in _pars:
            queue.append((par , dep))
    return list(ret)

In [6]:

def process_node(cat):
    cat = cat.replace(" ","_")
    if cat not in child_to_index : return
    parents[cat] = recurse(cat , set())

for cat in tqdm(data):
    process_node(cat)
print(len(parents))

100%|██████████| 34519/34519 [00:02<00:00, 16224.38it/s]

21651


In [8]:
from collections import defaultdict
cat_count = defaultdict(set)
for key , val in parents.items():
    key = key.replace("_" , " ")
    if key not in data_list_id : 
        continue
    sub_list = data_list_id[key]
    for _val in val:
        cat_count[_val[0]]|=set(sub_list)

In [9]:
cat_count = {key : val for key , val in sorted(cat_count.items() , key = lambda item : len(item[1]) , reverse = True)}

In [10]:
for key , val in list(cat_count.items())[:10]:
    print(key , len(val))

प्रमुख_विषय 78639
विज्ञान 69378
श्रेणियाँ 69004
समग्र_विषय 68568
विद्यार्जन_विषय 66196
प्रकृति 66050
लेख 65655
लक्षण_अनुसार_श्रेणियाँ 61798
सामाजिक_विज्ञान 61287
कॉमंस_श्रेणी_कड़ी_स्थानीय_तौर_पर_परिभाषित 61233


In [11]:
log = 'लोग'

In [12]:
with open('data_processed_singlechar.json') as f:
    data_single = json.loads(f.read())

In [13]:
import pandas as pd

In [14]:
df_log = pd.read_csv('people.csv')

In [15]:
petscan_log = df_log['pageid']

In [16]:
petscan_log = [ str(i) for i in petscan_log]

In [19]:
len(set(cat_count[log])|set(data_single[log])|set(petscan_log))

10467